In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_diabetes, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd

# Helper function to prepare datasets
def prepare_dataset(data, target, binary_classification=False):
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    y_train = torch.FloatTensor(y_train).reshape(-1, 1)
    y_test = torch.FloatTensor(y_test).reshape(-1, 1)
    
    # For binary classification, ensure targets are in binary format
    if binary_classification:
        y_train = y_train.round()  # Ensure targets are 0 or 1
        y_test = y_test.round()
    
    return torch.FloatTensor(X_train), torch.FloatTensor(X_test), y_train, y_test

# Load Diabetes Dataset
diabetes_data = load_diabetes()
X_diabetes, y_diabetes = diabetes_data.data, diabetes_data.target
# Convert target to binary for simplicity (above/below median)
median_target = y_diabetes.mean()
y_diabetes_binary = (y_diabetes > median_target).astype(float)
X_train_d, X_test_d, y_train_d, y_test_d = prepare_dataset(X_diabetes, y_diabetes_binary, binary_classification=True)

# Load Cancer Dataset
cancer_data = load_breast_cancer()
X_cancer, y_cancer = cancer_data.data, cancer_data.target
X_train_c, X_test_c, y_train_c, y_test_c = prepare_dataset(X_cancer, y_cancer, binary_classification=True)

# Load Sonar Dataset
sonar_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data', header=None)
X_sonar, y_sonar = sonar_data.iloc[:, :-1].values, pd.get_dummies(sonar_data.iloc[:, -1]).values[:, 1]
X_train_s, X_test_s, y_train_s, y_test_s = prepare_dataset(X_sonar, y_sonar, binary_classification=True)

# Define the Neural Network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_function):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.activation = activation_function
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x)  # Sigmoid for binary classification

# Training and Evaluation Function
def train_and_evaluate(X_train, X_test, y_train, y_test, activation_function, epochs=100, learning_rate=0.01):
    input_size = X_train.shape[1]
    hidden_size = 16
    output_size = 1

    model = NeuralNet(input_size, hidden_size, output_size, activation_function)
    criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        predictions = model(X_train)
        loss = criterion(predictions, y_train)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        predictions = (predictions > 0.5).float()
        accuracy = accuracy_score(y_test.numpy(), predictions.numpy())
        print(f"Test Accuracy: {accuracy:.4f}")

# Train on each dataset with different activation functions
print("Diabetes Dataset - Activation: ReLU")
train_and_evaluate(X_train_d, X_test_d, y_train_d, y_test_d, activation_function=nn.ReLU())

print("\nCancer Dataset - Activation: Tanh")
train_and_evaluate(X_train_c, X_test_c, y_train_c, y_test_c, activation_function=nn.Tanh())

print("\nSonar Dataset - Activation: Sigmoid")
train_and_evaluate(X_train_s, X_test_s, y_train_s, y_test_s, activation_function=nn.Sigmoid())


Diabetes Dataset - Activation: ReLU
Epoch 10/100, Loss: 0.5908
Epoch 20/100, Loss: 0.5295
Epoch 30/100, Loss: 0.4868
Epoch 40/100, Loss: 0.4712
Epoch 50/100, Loss: 0.4616
Epoch 60/100, Loss: 0.4544
Epoch 70/100, Loss: 0.4477
Epoch 80/100, Loss: 0.4415
Epoch 90/100, Loss: 0.4357
Epoch 100/100, Loss: 0.4300
Test Accuracy: 0.7416

Cancer Dataset - Activation: Tanh
Epoch 10/100, Loss: 0.2400
Epoch 20/100, Loss: 0.1136
Epoch 30/100, Loss: 0.0775
Epoch 40/100, Loss: 0.0645
Epoch 50/100, Loss: 0.0580
Epoch 60/100, Loss: 0.0541
Epoch 70/100, Loss: 0.0508
Epoch 80/100, Loss: 0.0479
Epoch 90/100, Loss: 0.0454
Epoch 100/100, Loss: 0.0432
Test Accuracy: 0.9825

Sonar Dataset - Activation: Sigmoid
Epoch 10/100, Loss: 0.5504
Epoch 20/100, Loss: 0.4380
Epoch 30/100, Loss: 0.3583
Epoch 40/100, Loss: 0.3005
Epoch 50/100, Loss: 0.2501
Epoch 60/100, Loss: 0.2082
Epoch 70/100, Loss: 0.1722
Epoch 80/100, Loss: 0.1421
Epoch 90/100, Loss: 0.1175
Epoch 100/100, Loss: 0.0980
Test Accuracy: 0.8095
